Ben Trevett 의 [Convolutional Sentiment Analysis](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/4%20-%20Convolutional%20Sentiment%20Analysis.ipynb) 튜토리얼을 한글 데이터셋에 적용해보는 연습이다. 데이터셋은 [네이버 영화 평점 데이터](https://github.com/e9t/nsmc)을 이용한다.

이 튜토리얼에서는 [Convolutional Neural Networks for Sentence Classification](https://arxiv.org/abs/1408.5882) 논문에서 제시된 CNN 기반 모델을 이용해서 감정 분석을 해보자. 일반적으로 CNN은 비전 관련 데이터를 처리할 때 사용되지만, 위 논문에서는 [1 x 2] 크기의 필터를 이용하여 bi-gram과 유사한 효과를 얻어내었다.

# 전처리

이 모델은 앞서 설명한 것처럼 CNN의 필터를 이용하기 때문에 `FastText`처럼 bi-gram 생성 함수를 이용할 필요가 없다. 우리는 한글 데이터를 다루므로 토크나이저 또한 별도로 지정해야한다. 여기서는 [KoNLPy](https://konlpy-ko.readthedocs.io/ko/v0.4.3/)의 은전한닢 tokenizer를 이용한다. CNN 모델은 지난 번에 설명한 것처럼 배치 사이즈를 첫번째 차원으로 받기 때문에 `batch_first = True` 옵션을 주면 된다. 

In [2]:
import torch
from torchtext import data
from torchtext import datasets
import random
import numpy as np
from konlpy.tag import Mecab
mecab = Mecab()

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

다만 문장의 길이가 필터 사이즈보다 작으면 에러가 나므로 다음과 같이 토크나이저를 수정하자.

In [40]:
FILTER_SIZES = [3,4,5]
def tokenizer(text):
    token = [t for t in mecab.morphs(text)]
    if len(token) < max(FILTER_SIZES):
        for i in range(0, max(FILTER_SIZES) - len(token)):
            token.append('<PAD>')
    return token

In [41]:
TEXT = data.Field(tokenize = tokenizer, batch_first = True)
LABEL = data.LabelField(dtype = torch.float)

In [42]:
fields = {'text': ('text',TEXT), 'label': ('label',LABEL)}
# dictionary 형식은 {csv컬럼명 : (데이터 컬럼명, Field이름)}

In [43]:
import random
train_data, test_data = data.TabularDataset.splits(
                            path = 'data',
                            train = 'train_data.csv',
                            test = 'test_data.csv',
                            format = 'csv',
                            fields = fields,  
)
train_data, valid_data = train_data.split(random_state=random.seed(SEED))

다음으로 단어 벡터는 전처리된 단어 벡터를 받자. 원 튜토리얼에선 `glove.100d`를 쓰지만 이건 한글을 지원하지 않으므로, 여기선 한글을 지원하는 `fasttext.simple.300d` 를 사용하겠다. 그리고 사전훈련된 단어집에 없는 단어는 0으로 처리하는 걸 방지하기 위해 `unk_init = torch.Tensor.normal_` 옵션을 준다. 

In [44]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data,
                max_size = MAX_VOCAB_SIZE,
                vectors = 'fasttext.simple.300d',
                unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

데이터 생성자를 만들자. 한글 데이터에선 오류가 발생해서 아래와 같이 `sort_key = lambda x: len(x.text)` 문장을 먼저 넣어줘야 오류없이 작동한다.

In [45]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.text),
    sort_within_batch = True,
    device = device)

# 모델 생성

여기서는 입력 문장을 임베딩 시킨 후 2차원 CNN을 다음과 같이 적용한다. 필터 사이즈는 [n x emb_dim] 이다.

<img src = 'https://github.com/bentrevett/pytorch-sentiment-analysis/raw/79bb86abc9e89951a5f8c4a25ca5de6a491a4f5d/assets/sentiment12.png'>

<img src = 'https://github.com/bentrevett/pytorch-sentiment-analysis/raw/79bb86abc9e89951a5f8c4a25ca5de6a491a4f5d/assets/sentiment13.png'>
     
<img src = 'https://github.com/bentrevett/pytorch-sentiment-analysis/raw/79bb86abc9e89951a5f8c4a25ca5de6a491a4f5d/assets/sentiment14.png'>    

이렇게 얻어진 벡터에 맥스 풀링(`F.max_pool1d`)을 적용한 후 `ReLU` 액티베이션을 적용한다. 

<img src = 'https://github.com/bentrevett/pytorch-sentiment-analysis/raw/79bb86abc9e89951a5f8c4a25ca5de6a491a4f5d/assets/sentiment15.png'>

다양한 사이즈의 필터를 적용하여 얻어진 벡터를 concatenate한 후 드랍아웃을 적용하고 마지막으로 Linear 층에 통과 시켜 output 을 산출한다.

In [46]:
import torch.nn as nn
import torch.nn.functional as F

여러개의 CNN 레이어를 리스트 형태로 생성하기 위해 `nn.ModuleList`을 이용하자.

사이즈 계산을 위한 함수를 사용하자.

In [47]:
def print_shape(name, data):
    print(f'{name} has shape {data.shape}')

In [48]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.convs = nn.ModuleList([nn.Conv2d(in_channels=1,
                                             out_channels=n_filters,
                                             kernel_size=(fs, embedding_dim))
                                   for fs in filter_sizes])
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        #print_shape('text', text)
        # text = [batch_size, sent_len]
        
        embedded = self.embedding(text)
        #print_shape('embedded', embedded)
        # embedded = [batch_size, sent_len, emb_dim]
        
        embedded = embedded.unsqueeze(1)
        #print_shape('embedded', embedded)
        # embedded = [batch_size, 1, sent_len, emb_dim]
        
        #print_shape('self.convs[0](embedded)', self.convs[0](embedded))
        # self.convs[0](embedded) = [batch_size, n_filters, sent_len-filter_sizes[n]+1, 1 ]
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
        
        #print_shape('F.max_pool1d(conved[0], conved[0].shape[2])', F.max_pool1d(conved[0], conved[0].shape[2]))
        # F.max_pool1d(conved[0], conved[0].shape[2]) = [batch_size, n_filters, 1]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        cat = self.dropout(torch.cat(pooled, dim=1))
        #print_shape('cat', cat)
        # cat = [batch_size, n_filters * len(filter_size)]
        
        res = self.fc(cat)
        #print_shape('res', res)
        # res = [batch_size, output_dim]
        
        return self.fc(cat)

In [49]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 1
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

모델의 벡터 사이즈 체크해보자.

In [50]:
model = model.to(device)

In [52]:
#inp = next(iter(train_iterator))
#model(inp.text)

모델의 파라미터 갯수는?

In [53]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'모델의 파라미터 수는 {count_parameters(model):,} 개 입니다.')

모델의 파라미터 수는 7,861,201 개 입니다.


사전 훈련된 단어 벡터를 불러오자.

In [54]:
pretrained_weight = TEXT.vocab.vectors
print(pretrained_weight.shape, model.embedding.weight.data.shape)

torch.Size([25002, 300]) torch.Size([25002, 300])


In [55]:
model.embedding.weight.data.copy_(pretrained_weight)

tensor([[-0.5461,  1.3300,  0.8290,  ...,  2.0847, -0.8228,  0.7651],
        [ 1.2100,  0.7778,  0.8799,  ...,  1.5097, -0.4671, -0.0241],
        [ 0.0569, -0.0520,  0.2733,  ..., -0.0695, -0.1606, -0.0989],
        ...,
        [ 0.6331,  0.1314, -0.9084,  ..., -1.8106,  1.8545, -1.4324],
        [-0.6003,  2.2804,  0.3818,  ..., -0.3988, -0.4746, -0.7577],
        [ 0.4369,  0.2952,  0.6235,  ...,  1.0262,  0.6049, -0.2220]],
       device='cuda:0')

`UNK_IDX`와 `PAD_IDX`는 제로 처리한다.

In [56]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

# 모델 훈련

이전과 동일하게 하자

In [57]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [58]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [59]:
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds==y).float()
    acc = correct.sum() / len(correct)
    return acc

훈련 함수를 정의하자. 여기선 드랍아웃 안쓰지만 걍 `model.train()` 사용하겠다.

In [60]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        predictions = model(batch.text).squeeze(1) # output_dim = 1
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [61]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

얼마나 훈련 걸리는 지 체크하는 함수

In [62]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

이제 훈련시켜보자.


In [63]:
N_EPOCHS = 5
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut4-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 27s
	Train Loss: 0.447 | Train Acc: 78.65%
	 Val. Loss: 0.368 |  Val. Acc: 83.63%
Epoch: 02 | Epoch Time: 0m 27s
	Train Loss: 0.338 | Train Acc: 85.49%
	 Val. Loss: 0.342 |  Val. Acc: 85.25%
Epoch: 03 | Epoch Time: 0m 27s
	Train Loss: 0.285 | Train Acc: 88.29%
	 Val. Loss: 0.339 |  Val. Acc: 85.72%
Epoch: 04 | Epoch Time: 0m 27s
	Train Loss: 0.243 | Train Acc: 90.30%
	 Val. Loss: 0.355 |  Val. Acc: 85.77%
Epoch: 05 | Epoch Time: 0m 27s
	Train Loss: 0.205 | Train Acc: 92.03%
	 Val. Loss: 0.381 |  Val. Acc: 85.42%


나쁘지 않군! 게다가 훈련 시간은 1/4 이다.

테스트셋에서 돌려보자.

In [64]:
model.load_state_dict(torch.load('tut4-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.344 | Test Acc: 85.58%


더 훈련시켜보자.

In [65]:
for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut4-model.pt')
    
    print(f'Epoch: {epoch+6:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 06 | Epoch Time: 0m 27s
	Train Loss: 0.241 | Train Acc: 90.38%
	 Val. Loss: 0.369 |  Val. Acc: 85.30%
Epoch: 07 | Epoch Time: 0m 27s
	Train Loss: 0.205 | Train Acc: 91.98%
	 Val. Loss: 0.385 |  Val. Acc: 85.44%
Epoch: 08 | Epoch Time: 0m 28s
	Train Loss: 0.173 | Train Acc: 93.37%
	 Val. Loss: 0.427 |  Val. Acc: 85.27%
Epoch: 09 | Epoch Time: 0m 28s
	Train Loss: 0.145 | Train Acc: 94.51%
	 Val. Loss: 0.488 |  Val. Acc: 85.09%
Epoch: 10 | Epoch Time: 0m 27s
	Train Loss: 0.127 | Train Acc: 95.32%
	 Val. Loss: 0.534 |  Val. Acc: 84.93%


오버피팅이 발생하고 있다...

In [66]:
model.load_state_dict(torch.load('tut4-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.344 | Test Acc: 85.58%


성능은 이전 모델과 거의 비슷하지만 훈련 시간이 대폭 감소!